In [1]:
import tensorflow as tf
from flows import NormalRW, DFlow, NVPFlow, LogNormal, GVAR, phase,\
Normal, floatX, MVNormal, MVNormalRW, Linear, LinearChol
import flows

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.contrib.distributions import WishartCholesky
import math

np.random.seed(1234)

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
data = pd.read_csv('./data/test_rus_data.csv').values.astype(floatX).T[np.newaxis][:,1:]/100

In [3]:
data = data[:,1:] - data[:,:-1]

In [4]:
stds = (data[0,1:] - data[0,:-1]).std(axis=0)
print(stds)
data /= stds

[0.43681782 0.25903013 0.01298369]


In [5]:
np.any(np.isnan(data))

False

In [6]:
NUM_STEPS = data.shape[1]

with tf.variable_scope('walk_ord'):
    s1_prior_d = LogNormal(1, mu=math.log(0.01), sigma=1.)

    with tf.variable_scope('s1_inference', dtype=floatX):
        mu = tf.get_variable('mu', shape=[1], initializer=tf.constant_initializer(s1_prior_d.mu))
        logsigma = tf.get_variable('logsigma', shape=[1], 
                                   initializer=tf.constant_initializer(math.log(s1_prior_d.sigma) - 1.))
        sigma = tf.exp(logsigma)
        s1_d = LogNormal(1, mu=mu, sigma=sigma)
    
    s1 = s1_d.sample()
    tf.summary.scalar('s1_ord', s1[0])
    tf.add_to_collection('logdensities', s1_d.logdens(s1))
    
    s1_prior = s1_prior_d.logdens(s1)
    tf.add_to_collection('priors', s1_prior)
    
#distribution for shock increase factor
with tf.variable_scope('walk_shock'):
    incr_prior_d = LogNormal(1, mu=math.log(1.), sigma=.5)

    with tf.variable_scope('incr_inference', dtype=floatX):
        mu = tf.get_variable('mu', shape=[1], initializer=tf.constant_initializer(incr_prior_d.mu))
        logsigma = tf.get_variable('logsigma', shape=[1], 
                                   initializer=tf.constant_initializer(math.log(incr_prior_d.sigma) - 1.))
        sigma = tf.exp(logsigma)
        incr_d = LogNormal(1, mu=mu, sigma=sigma)

    incr = incr_d.sample()
    tf.add_to_collection('logdensities', incr_d.logdens(incr))
    
    incr_prior = incr_prior_d.logdens(incr)
    tf.add_to_collection('priors', incr_prior)

    incr += 1.
    s_shock = s1*incr
    tf.summary.scalar('s1_shock', s_shock[0])
    
s0 = 80.
dim = [3,4]

K = dim[0] * dim[1]

PWalk = NormalRW(dim=None, sigma0=s0, sigma=s1, name='OrdWalk')
PWalk_shock = NormalRW(dim=None, sigma0=s0, sigma=s_shock, name='ShockWalk')

In [7]:
#model coefficients
gvar = GVAR(dim=dim[0]*dim[1], len=NUM_STEPS, name='coef_rw_inference')
outputs = gvar.sample()
# with tf.variable_scope('coefs_inference'):
#     gvar = DFlow([LinearChol(NUM_STEPS*dim[0]*dim[1], name='coef_rw_inference', use_bias=False)], init_sigma=0.02)
#     outputs = tf.reshape(gvar.output, [1,NUM_STEPS, dim[0]*dim[1]])
#     outputs = tf.cumsum(outputs, axis=1)
#     mu = tf.get_variable('mu', [1,NUM_STEPS,dim[0]*dim[1]])
#     outputs += mu

tf.add_to_collection('logdensities', gvar.logdens)

In [8]:
outputs

<tf.Tensor 'coef_rw_inference_1/VAR/strided_slice_20:0' shape=(1, 20, 12) dtype=float32>

In [9]:
prior_ord = PWalk.logdens(outputs, reduce=False)
prior_shock = PWalk_shock.logdens(outputs, reduce=False)

In [10]:
prior_ord, prior_shock

(<tf.Tensor 'OrdWalk_2/concat:0' shape=(1, 20) dtype=float32>,
 <tf.Tensor 'ShockWalk_2/concat:0' shape=(1, 20) dtype=float32>)

In [11]:
with tf.variable_scope('Shock_Distr', reuse=tf.AUTO_REUSE):
    shock_init_d = Normal([1,8])
    shock_init = shock_init_d.sample()
    
    tf.add_to_collection('logdensities', shock_init_d.logdens(shock_init))
    
    shock_d = flows.dvae.RMultinomial([data.shape[1], 2], name='shock_distr')
    shocks_soft_uncond, shocks_soft_cond, (shocks_hard, shocks_hard_logp, encoding_entropy) =\
    shock_d.encode(shock_init)
    
    shocks_hard_logp = tf.reduce_sum(shocks_hard_logp)

In [12]:
shock_d.temp

<tf.Tensor 'Shock_Distr/shock_distr/controls/Exp:0' shape=(20, 1) dtype=float32>

In [13]:
shocks_hard

<tf.Tensor 'Shock_Distr/shock_distr_1/encoder/latent_inf/Multinomial_1/sample/Reshape_3:0' shape=(1, 20, 2) dtype=float32>

In [15]:
def create_hard_shock_prior(num_shocks):
    def c(n,k):
        from scipy.special import binom
        return tf.py_func(lambda x,y: int(binom(int(x),int(y))), [n, k], tf.int64)
    
    with tf.variable_scope('shocks_prior'):
        spd = tf.contrib.distributions.Poisson(rate=NUM_STEPS/1.)
        denum = tf.cast(c(NUM_STEPS, num_shocks), tf.float32)
        return (spd.log_prob(num_shocks)) - tf.log(denum)

In [16]:
def create_soft_shock_prior(num_shocks):
    lower = tf.cast(tf.floor(num_shocks), floatX)
    upper = lower + 1.
    lh = create_hard_shock_prior(lower)
    uh = create_hard_shock_prior(upper)
    prop = num_shocks-lower
    return lh*(1.-prop) + uh*prop

In [17]:
obs_d_prior = LogNormal(dim=None, mu=math.log(0.5), sigma=0.5)

with tf.variable_scope('obs_d_inference', dtype=floatX):
    mu = tf.get_variable('mu', shape=[1], initializer=tf.constant_initializer(math.log(0.5)))
    logsigma = tf.get_variable('logsigma', shape=[1], initializer=tf.constant_initializer(-5))
    sigma = tf.exp(logsigma)
    
    obs_d_post = LogNormal(1, mu=mu, sigma=sigma)
    obs_ds = obs_d_post.sample()
    
    obs_ds_logdens = obs_d_post.logdens(obs_ds)
    tf.add_to_collection('logdensities', obs_ds_logdens)
    tf.add_to_collection('priors', obs_d_prior.logdens(obs_ds))
    
    tf.summary.scalar('observ_sigma', obs_ds[0])

In [18]:
def create_param_walk_prior(shocks):
    with tf.name_scope('RW_prior'):
        tmp = tf.cast(shocks[:,:,1], floatX)
        prior = tmp*prior_shock + (1-tmp)*prior_ord
        prior = tf.reduce_sum(prior) 
        return prior

In [19]:
def predict(observable_mask):
    out = tf.reshape(outputs, [NUM_STEPS, dim[0], dim[1]])
    
    def step(prev, x):
        mask = x[0]
        prev_pred = tf.where(mask, x[1], prev)[tf.newaxis]
        params = x[2]
        
        d = params[:,:-1]
        new_pred = tf.matmul(prev_pred, d)[0] + params[:,-1]
        return new_pred
    
    ar = tf.scan(step, [observable_mask, data[0], out], initializer=tf.zeros([dim[0]], dtype=floatX))
    return ar

In [20]:
data.shape[1]

20

In [21]:
stop_time = tf.placeholder_with_default(NUM_STEPS, ())
predict_mask = tf.range(0, data.shape[1], dtype=tf.int32) < stop_time
#используем в предсказаниях ровно stop_time последних наблюдений

In [22]:
preds = predict(predict_mask)

In [23]:
preds

<tf.Tensor 'scan/TensorArrayStack/TensorArrayGatherV3:0' shape=(20, 3) dtype=float32>

In [24]:
obs_d = Normal(dim=None, sigma=obs_ds, mu=0)

diffs = preds[:-1] - data[0,1:]

logl = obs_d.logdens(diffs, reduce=False)
logl *= tf.cast(predict_mask[:-1], floatX)[:,tf.newaxis]

logl = tf.reduce_sum(logl)
tf.add_to_collection('priors', logl)

In [25]:
tf.summary.scalar('logl', logl)

<tf.Tensor 'logl:0' shape=() dtype=string>

In [26]:
def create_kl(shocks, shocks_prior=0.):
    logdens = tf.reduce_sum(tf.get_collection('logdensities'))
    prior = tf.reduce_sum(tf.get_collection('priors'))
    
    param_walk_prior = create_param_walk_prior(shocks)
    KLd = -prior + logdens - shocks_prior - param_walk_prior
    KLd /= NUM_STEPS*dim[0]*dim[1]
    return KLd

In [27]:
tf.get_collection('logdensities')

[<tf.Tensor 'walk_ord/LogNormal_2/Sum:0' shape=() dtype=float32>,
 <tf.Tensor 'walk_shock/LogNormal_2/Sum:0' shape=() dtype=float32>,
 <tf.Tensor 'coef_rw_inference_1/VAR/logdens/add:0' shape=() dtype=float32>,
 <tf.Tensor 'Shock_Distr/Normal_2/Sum:0' shape=() dtype=float32>,
 <tf.Tensor 'obs_d_inference/LogNormal_2/Sum:0' shape=() dtype=float32>]

In [28]:
tf.get_collection('priors')

[<tf.Tensor 'walk_ord/LogNormal_3/Sum:0' shape=() dtype=float32>,
 <tf.Tensor 'walk_shock/LogNormal_3/Sum:0' shape=() dtype=float32>,
 <tf.Tensor 'obs_d_inference/LogNormal_3/Sum:0' shape=() dtype=float32>,
 <tf.Tensor 'Sum:0' shape=() dtype=float32>]

In [29]:
shocks_soft_cond

<tf.Tensor 'Shock_Distr/shock_distr_1/encoder/latent_inf/Reshape_4:0' shape=(1, 20, 2) dtype=float32>

In [30]:
num_shocks = tf.reduce_sum(shocks_soft_uncond[:,:,1:])
ssp = create_soft_shock_prior(num_shocks) #- encoding_entropy
soft_kl_uncond = create_kl(shocks_soft_uncond, shocks_prior=ssp)

num_shocks = tf.reduce_sum(shocks_soft_cond[:,:,1:])
ssp = create_soft_shock_prior(num_shocks) #- encoding_entropy
soft_kl_cond = create_kl(shocks_soft_cond, shocks_prior=ssp)

num_shocks = tf.reduce_sum(shocks_hard[:,:,1:])
hard_shocks_prior = create_hard_shock_prior(num_shocks) - shocks_hard_logp
hard_kl = create_kl(shocks_hard, shocks_prior=hard_shocks_prior)

In [31]:
tf.summary.scalar('num_shocks', num_shocks)

<tf.Tensor 'num_shocks:0' shape=() dtype=string>

In [32]:
shocks_hard_logp

<tf.Tensor 'Shock_Distr/Sum:0' shape=() dtype=float32>

In [33]:
relax_cond = shock_d.build_relax(tf.reshape(shocks_soft_cond, [1,-1]))
relax_uncond = shock_d.build_relax(tf.reshape(shocks_soft_uncond, [1,-1]))

In [34]:
soft_kl_cond

<tf.Tensor 'truediv_1:0' shape=<unknown> dtype=float32>

In [35]:
eta = shock_d.eta

soft_control_cond = eta*soft_kl_cond + relax_cond
soft_control_uncond = eta*soft_kl_uncond - eta*soft_kl_cond - relax_cond + relax_uncond

stoped_grad = hard_kl - soft_control_cond# - nvil_baseline

target = shocks_hard_logp*stoped_grad + soft_control_uncond
target = tf.reduce_mean(target)

In [36]:
soft_control_uncond

<tf.Tensor 'add_8:0' shape=<unknown> dtype=float32>

In [37]:
shock_vars = shock_d.get_encoder_vars()

In [38]:
shock_vars_grad = tf.gradients(target, shock_vars, stop_gradients=stoped_grad)

In [39]:
controls = shock_d.get_control_vars()
cv_loss = shock_d.build_control_loss(shock_vars_grad)

tf.summary.scalar('cv_loss', cv_loss)

cv_gradients = tf.gradients(cv_loss, controls)

In [40]:
rest_vars = [x for x in tf.global_variables() if not (x in controls or x in shock_vars)]
rest_gradients = tf.gradients(hard_kl, rest_vars)

In [41]:
rest_vars

[<tf.Variable 'walk_ord/s1_inference/mu:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'walk_ord/s1_inference/logsigma:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'walk_shock/incr_inference/mu:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'walk_shock/incr_inference/logsigma:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'coef_rw_inference/lc_0/lowerd:0' shape=(78,) dtype=float32_ref>,
 <tf.Variable 'coef_rw_inference/lc_0/ldiag:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'coef_rw_inference/lc_1/lowerd:0' shape=(78,) dtype=float32_ref>,
 <tf.Variable 'coef_rw_inference/lc_1/ldiag:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'coef_rw_inference/lc_1/W:0' shape=(12, 12) dtype=float32_ref>,
 <tf.Variable 'coef_rw_inference/lc_2/lowerd:0' shape=(78,) dtype=float32_ref>,
 <tf.Variable 'coef_rw_inference/lc_2/ldiag:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'coef_rw_inference/lc_2/W:0' shape=(12, 12) dtype=float32_ref>,
 <tf.Variable 'coef_rw_inference/lc_3/lowerd:0' sha

In [42]:
all_vars = shock_vars + rest_vars 
grads = shock_vars_grad + rest_gradients
main_updates = [(g,x) for x,g in zip(all_vars, grads)]
control_updates = [(g,x) for x,g in zip(controls, cv_gradients)]

In [43]:
tf.summary.scalar('KLd', hard_kl)

<tf.Tensor 'KLd:0' shape=() dtype=string>

In [44]:
lr = tf.Variable(0.001)
main_opt = tf.train.AdamOptimizer(lr).apply_gradients(main_updates)
cont_opt = tf.train.AdadeltaOptimizer(0.001).apply_gradients(control_updates)

In [45]:
control_updates

[(<tf.Tensor 'gradients_1/Shock_Distr/shock_distr/controls/Exp_grad/mul:0' shape=(20, 1) dtype=float32>,
  <tf.Variable 'Shock_Distr/shock_distr/controls/pretemp:0' shape=(20, 1) dtype=float32_ref>),
 (<tf.Tensor 'gradients_1/Shock_Distr/shock_distr/controls/Exp_1_grad/mul:0' shape=() dtype=float32>,
  <tf.Variable 'Shock_Distr/shock_distr/controls/preeta:0' shape=() dtype=float32_ref>),
 (<tf.Tensor 'gradients_1/AddN_30:0' shape=(40, 128) dtype=float32>,
  <tf.Variable 'Shock_Distr/shock_distr/controls/RELAX/FCN/d0/W:0' shape=(40, 128) dtype=float32_ref>),
 (<tf.Tensor 'gradients_1/AddN_29:0' shape=(1, 128) dtype=float32>,
  <tf.Variable 'Shock_Distr/shock_distr/controls/RELAX/FCN/d0/b:0' shape=(1, 128) dtype=float32_ref>),
 (<tf.Tensor 'gradients_1/AddN_24:0' shape=(128, 64) dtype=float32>,
  <tf.Variable 'Shock_Distr/shock_distr/controls/RELAX/FCN/d1/W:0' shape=(128, 64) dtype=float32_ref>),
 (<tf.Tensor 'gradients_1/AddN_22:0' shape=(1, 64) dtype=float32>,
  <tf.Variable 'Shock_Dis

In [46]:
opt = [main_opt, cont_opt]

In [47]:
sess = tf.InteractiveSession()

In [48]:
# !rm -R /tmp/tfdbg

In [49]:
!mkdir /tmp/tfdbg
writer = tf.summary.FileWriter('/tmp/tfdbg/relax_rus_var_scaledprior_novae_better_prior_fully_proper_highrate')

mkdir: cannot create directory ‘/tmp/tfdbg’: File exists


In [50]:
writer.add_graph(tf.get_default_graph())

In [51]:
tf.global_variables_initializer().run()

In [64]:
hard_kl.eval()

nan

In [53]:
cv_gradients[0]

<tf.Tensor 'gradients_1/Shock_Distr/shock_distr/controls/Exp_grad/mul:0' shape=(20, 1) dtype=float32>

In [54]:
# a = []
# for _ in range(1000):
#     a.append(sess.run(cv_gradients[0]))
# np.mean(a, axis=0)

In [55]:
np.set_printoptions(precision=3)

In [56]:
# PWalk.inverse_sigma.eval()[:3,:3]

In [57]:
out = tf.reshape(outputs, [NUM_STEPS, dim[0], dim[1]])

In [58]:
epoch = 0

In [59]:
all_sum = tf.summary.merge_all()

In [60]:
def check_ppc(timestep):
    n = 1000
    all_preds = []
    for _ in range(n):
        tp = preds.eval({stop_time:timestep-1})
        tp = tp[timestep-1]
        all_preds.append(tp)
    all_preds = np.array(all_preds).mean(axis=0)
#     print(xs.shape)
    return np.sqrt(np.mean((all_preds - xs[0,timestep])**2))

In [61]:
# shock_d.logits.eval()

In [62]:
%%time
for epoch in range(epoch, epoch+1000):
    for i in range(1000):
        l, _ = sess.run([hard_kl, opt], {phase:True})
        if i % 30 == 0:
            s = all_sum.eval({phase:True})
            writer.add_summary(s, global_step=epoch*1000 + i)
    print(l)

0.7502741
0.60144526
0.5530164
0.5771306
0.54054314
0.54968745
0.4952123
0.5186384
0.49267933
0.5269511
0.51685387
0.5165606
0.50924456
0.49109623
0.5088951
0.5141146
0.5272558
0.53858364
0.53418297
0.54433084
0.540536
0.52195203
0.5207203
0.51618016


InvalidArgumentError: ValueError: cannot convert float NaN to integer
Traceback (most recent call last):

  File "/usr/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 147, in __call__
    ret = func(*args)

  File "<ipython-input-15-981260308877>", line 4, in <lambda>
    return tf.py_func(lambda x,y: int(binom(int(x),int(y))), [n, k], tf.int64)

ValueError: cannot convert float NaN to integer


	 [[Node: shocks_prior_2/PyFunc = PyFunc[Tin=[DT_INT32, DT_FLOAT], Tout=[DT_INT64], token="pyfunc_2", _device="/job:localhost/replica:0/task:0/device:CPU:0"](shocks_prior_2/PyFunc/input_0, Floor_1)]]

Caused by op 'shocks_prior_2/PyFunc', defined at:
  File "/usr/lib64/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib64/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/nikita/.virtualenvs/py3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instan

In [62]:
lr.assign(0.00005).eval()

5e-05

In [63]:
ss = []
for _ in range(40000):
    s = shocks_hard.eval()[:,:,1].astype('float')
    ss.append(s)
ss = np.array(ss)
print(ss.mean(axis=0))

KeyboardInterrupt: 

In [ ]:
x = range(2016-data.shape[1],2016)
plt.plot(x, ss.mean(axis=0)[0][:])
plt.show()

In [ ]:
ss = []
aa = []
for _ in range(3000):
    a = np.linalg.inv(PWalk.inverse_sigma.eval())
    ss.append(np.sqrt(np.diag(a)))
    aa.append(a)
ss = np.array(ss)
aa = np.array(aa)

In [ ]:
aa.mean(axis=0)[:5,:5]

In [ ]:
np.mean(ss, axis=0), np.std(ss, axis=0)

In [ ]:
# sns.kdeplot(ss[:,1], ss[:,0])
# plt.show()

In [ ]:
samples = []
for _ in range(3000):
    s = out.eval({phase:True})
    samples.append(s)
samples = np.array(samples)

In [ ]:
t = 10
sns.kdeplot(samples[:,t,2,0], samples[:,t,0,3])
plt.show()

In [ ]:
t = 10
# params[t]

In [ ]:
np.mean(samples[:,t], axis=0)

In [ ]:
np.std(samples[:,t], axis=0)

In [ ]:
from sklearn.neighbors.kde import KernelDensity
def score(s1, s2):
    bw = 0.1
    s1 = s1[:,np.newaxis]
    s2 = s2[:,np.newaxis]
    e1 = KernelDensity(bandwidth=bw).fit(s1)
    e2 = KernelDensity(bandwidth=bw).fit(s2)
    
    e1e2 = e1.score(s1) - e2.score(s1)
    e2e1 = e2.score(s2) - e1.score(s2)
    dist = (e1e2/len(s1) + e2e1/len(s2))/2
    return dist

In [ ]:
def get_cdf(ts):
    return lambda x: (ts <= x).mean()
def score(s1,s2):
    c1 = get_cdf(s1)
    c2 = get_cdf(s2)

    m = 0
    for s in np.concatenate([s1, s2]):
        r = np.abs(c1(s) - c2(s))
        if r > m:
            m = r
    return m

In [ ]:
score(s1,s2)

In [ ]:
dists = []
for t1 in range(samples.shape[1] - 1):
    np.random.seed(1234)
    print(t1)
    t2 = t1 + 1
    s1 = samples[:,t1,0,0]
    s2 = samples[:,t2,0,0]
    ss = []
    for _ in range(10):
        s1_ = np.random.choice(s1, size=8000)
        s2_ = np.random.choice(s2, size=8000)
        m = score(s1_,s2_)
        ss.append(m)
    dists.append(ss)
dists = np.array(dists).T

## KDE + KL

In [ ]:
sns.tsplot(dists, ci=[50, 95])
plt.show()

In [ ]:
sns.tsplot(dists, ci=[50, 95])
plt.show()

## Kolmogorov-Smirnov

In [ ]:
sns.tsplot(dists, ci=[50, 95])
plt.show()

In [ ]:
tf.pow()